In [4]:
from pyspark import SparkConf, SparkContext

In [5]:
config = SparkConf().setAppName("learning").setMaster("local")

In [6]:
sc = SparkContext(conf=config)

# 5) Find the employee count & cost to company for each group consisting of dept, cadre, and state? Filename: sales.txt
<pre>
Output:
dept,cadre,sate,TotalEmployeeCount,costToCompany
((Sales,Trainee,UK),(1,12000))
((Sales,Lead,IND),(2,64000))
((Marketing,Associate,IND),(2,36000))
((Sales,Lead,AUS),(2,64000))
((HR,Manager,IND),(1,58000))
((Sales,Lead,NY),(3,96000))
</pre>

In [9]:
sales_rdd = sc.textFile("file:///home/saif/LFS/datasets/sales.txt")

In [10]:
sales_rdd.take(5)

['dept,cadre,costToCompany,state',
 'Sales,Trainee,12000,UK',
 'Sales,Lead,32000,AUS',
 'Sales,Lead,32000,NY',
 'Sales,Lead,32000,IND']

In [12]:
header = sales_rdd.first()

In [13]:
sales_rdd_without_header = sales_rdd.filter(lambda x : x!=header)

In [14]:
sales_rdd_without_header = sales_rdd_without_header.map(lambda x: x.split(","))

In [15]:
sales_rdd_without_header.take(5)

[['Sales', 'Trainee', '12000', 'UK'],
 ['Sales', 'Lead', '32000', 'AUS'],
 ['Sales', 'Lead', '32000', 'NY'],
 ['Sales', 'Lead', '32000', 'IND'],
 ['Sales', 'Lead', '32000', 'AUS']]

In [19]:
sales_rdd_without_header_grouping= sales_rdd_without_header.map(lambda row:((row[0],row[1],row[3]),(1,int(row[2]))))

In [20]:
sales_rdd_without_header_grouping.take(5)

[(('Sales', 'Trainee', 'UK'), (1, 12000)),
 (('Sales', 'Lead', 'AUS'), (1, 32000)),
 (('Sales', 'Lead', 'NY'), (1, 32000)),
 (('Sales', 'Lead', 'IND'), (1, 32000)),
 (('Sales', 'Lead', 'AUS'), (1, 32000))]

In [21]:
sales_rdd_without_header_grouping = sales_rdd_without_header_grouping.reduceByKey(lambda a,b:(a[0]+b[0],a[1]+b[1]))

In [22]:
sales_rdd_without_header_grouping.take(5)

[(('Sales', 'Trainee', 'UK'), (1, 12000)),
 (('Sales', 'Lead', 'AUS'), (2, 64000)),
 (('Sales', 'Lead', 'NY'), (3, 96000)),
 (('Sales', 'Lead', 'IND'), (2, 64000)),
 (('Marketing', 'Associate', 'IND'), (2, 36000))]

In [23]:
headerList = header.split(',')
print(f"{headerList[0]} {headerList[1]} {headerList[3]} TotalEmployeeCount costToCompany ")
for i in sales_rdd_without_header_grouping.collect():
    print(i)

dept cadre state TotalEmployeeCount costToCompany 
(('Sales', 'Trainee', 'UK'), (1, 12000))
(('Sales', 'Lead', 'AUS'), (2, 64000))
(('Sales', 'Lead', 'NY'), (3, 96000))
(('Sales', 'Lead', 'IND'), (2, 64000))
(('Marketing', 'Associate', 'IND'), (2, 36000))
(('HR', 'Manager', 'IND'), (1, 58000))
